In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import google.generativeai as genai
from langchain_pinecone import PineconeVectorStore
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
from pinecone import Pinecone
from config import source_file_path
from config import gemini_api_key
from config import embeddings_model
from config import llm_model
from config import pc_api
from config import index_name

In [2]:
page_content=f"Hello, my internet has been extremely slow for the past few days. I’m paying for high-speed service, but I can barely load a webpage! I’m sorry to hear that, sir. I understand how frustrating slow internet can be. Could you please provide your account number so I can check your connection? Sure, it’s 123456. Thank you. I see that there is an issue with the network in your area. Our technical team is already working on it, and the service should be restored within 24 hours. 24 hours? That’s unacceptable! I work from home and need the internet. I completely understand your concern. While the issue is being fixed, I can offer you 10GB of mobile data for free so you can continue working. Would that help? That would be helpful, yes. Great, I’ll activate it right away. You’ll receive a confirmation message shortly. I sincerely apologize for the inconvenience."

In [3]:
from langchain.schema import Document

doc = [Document(
    page_content=page_content,
    metadata={"source": source_file_path, "page": 1}
)]

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=50)

In [5]:
documents = text_splitter.split_documents(doc)

In [6]:
genai.configure(api_key=gemini_api_key)

In [7]:
embeddings=GoogleGenerativeAIEmbeddings(model=embeddings_model,google_api_key=gemini_api_key)

In [8]:
os.environ["PINECONE_API_KEY"] = pc_api

In [9]:
PineconeVectorStore.from_documents(documents, embeddings, index_name=index_name)

In [14]:
pc = Pinecone(api_key=pc_api)
index = pc.Index(index_name)

In [21]:
model_llm=genai.GenerativeModel(llm_model)

generation_config=genai.GenerationConfig(
    max_output_tokens=1000,
    temperature=0.1
)

In [27]:
def generate_ans(Question):
    Q_embedding=genai.embed_content(
        model=embeddings_model,
        content=Question)
    
    xc = index.query(vector=Q_embedding['embedding'], top_k=1, include_metadata=True)
    
    context=xc.matches[0]['metadata']['text']
    print("Context:\n",context)
    
    message=f"""
    Your task is to provide answer of the question from the context provided.
    Question:
    {Question}
    Context:
    {context}
    """
    
    response=model_llm.generate_content(message,generation_config=generation_config)
    answer=response.text
    print("Answer:\n",answer)
    return answer

In [28]:
Question=f"What is customer's complaint?"
generate_ans(Question)

Context:
 Hello, my internet has been extremely slow for the past few days. I’m paying for high-speed service, but I can barely load a webpage! I’m sorry to hear that, sir. I understand how frustrating slow internet can be. Could you please provide your account number so I can check your connection? Sure, it’s 123456. Thank you. I see that there is an issue with the network in your area. Our technical team is already working on it, and the service should be restored within 24 hours. 24 hours? That’s unacceptable! I work from home and need the internet. I completely understand your concern. While the
Answer:
 The customer's complaint is that their internet has been extremely slow for the past few days, despite paying for high-speed service.  They can barely load a webpage and this is impacting their ability to work from home.



"The customer's complaint is that their internet has been extremely slow for the past few days, despite paying for high-speed service.  They can barely load a webpage and this is impacting their ability to work from home.\n"

In [29]:
Question=f"Why the customer is fed up?"
generate_ans(Question)

Context:
 Hello, my internet has been extremely slow for the past few days. I’m paying for high-speed service, but I can barely load a webpage! I’m sorry to hear that, sir. I understand how frustrating slow internet can be. Could you please provide your account number so I can check your connection? Sure, it’s 123456. Thank you. I see that there is an issue with the network in your area. Our technical team is already working on it, and the service should be restored within 24 hours. 24 hours? That’s unacceptable! I work from home and need the internet. I completely understand your concern. While the
Answer:
 The customer is fed up because their internet has been extremely slow for several days, despite paying for high-speed service.  This is impacting their ability to work from home.



'The customer is fed up because their internet has been extremely slow for several days, despite paying for high-speed service.  This is impacting their ability to work from home.\n'

In [30]:
Question=f"Why the customer is frustrated?"
generate_ans(Question)

Context:
 Hello, my internet has been extremely slow for the past few days. I’m paying for high-speed service, but I can barely load a webpage! I’m sorry to hear that, sir. I understand how frustrating slow internet can be. Could you please provide your account number so I can check your connection? Sure, it’s 123456. Thank you. I see that there is an issue with the network in your area. Our technical team is already working on it, and the service should be restored within 24 hours. 24 hours? That’s unacceptable! I work from home and need the internet. I completely understand your concern. While the
Answer:
 The customer is frustrated because their internet has been extremely slow for several days, despite paying for high-speed service.  This is impacting their ability to work from home.  The 24-hour estimated repair time is also unacceptable to them.



'The customer is frustrated because their internet has been extremely slow for several days, despite paying for high-speed service.  This is impacting their ability to work from home.  The 24-hour estimated repair time is also unacceptable to them.\n'